In [ ]:
%%capture
!pip install transformers
!pip install spacy-cleaner

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path
import nltk
import spacy
from transformers import pipeline
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators
from gensim.parsing.preprocessing import remove_stopwords
import gensim

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ModuleNotFoundError: ignored

In [ ]:
gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS

In [ ]:

import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/all_news.xlsx')
df['title']=df['title'].fillna('')
df['text_all']=df['title']+' '+df['text']

In [ ]:
# TT
tmp0=pd.read_csv('/content/drive/My Drive/Colab Notebooks/idub_mk_tt/all0.csv')
tmp1=pd.read_csv('/content/drive/My Drive/Colab Notebooks/idub_mk_tt/all.csv')

In [ ]:
len(tmp0)

95414

In [ ]:
tmp=pd.concat([tmp0,tmp1])

In [ ]:
tmp=tmp.drop_duplicates(subset='url')

In [ ]:
len(tmp)

146553

In [ ]:
tmp=tmp[['text','created_at','umb','context_annotations','public_metrics','url']]

In [ ]:
# df['text_all'].str.contains('accessibility')

2314

In [ ]:
df['tech']=df['text_all'].str.contains('accessibility')

In [ ]:
df1=df[df['tech']==True]

In [ ]:
df1.reset_index(inplace=True)

In [ ]:
len(df1)

2314

In [ ]:
# df1['text_all'][1]

In [ ]:
df1=df1.merge(tmp,on='url',suffixes=['','_tweet'])

In [ ]:
df20=df1[df1['created_at']<'2021-12-31'].sample(10)
df22=df1[df1['created_at']>'2022-01-01'].sample(10)

In [ ]:
# len(df22)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', top_k=None)
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', top_k=None)

In [ ]:
docs20=[]
for idx, row in df20.iterrows():
  if idx%100==0:
    print(idx)
  docs20.append(nlp(row['text_all']))

In [ ]:
docs22=[]
for idx, row in df22.iterrows():
  if idx%100==0:
    print(idx)
  docs22.append(nlp(row['text_all']))

In [ ]:
sentences20=[]
for doc in docs20:
  tmplist=[]
  for sent in doc.sents:
    tmplist.append(sent.text)
  sentences20.append(tmplist)

In [ ]:
sentences22=[]
for doc in docs22:
  tmplist=[]
  for sent in doc.sents:
    tmplist.append(sent.text)
  sentences22.append(tmplist)

In [ ]:
sentsClean20=[]

for sent in sentences20:
  tmplist=[]
  for s in sent:
    clean=remove_stopwords(s.lower())
    if len(clean)>30:
      tmplist.append(clean)
    else:
      continue
  sentsClean20.append(tmplist)

In [ ]:
sentsClean22=[]

for sent in sentences22:
  tmplist=[]
  for s in sent:
    clean=remove_stopwords(s.lower())
    if len(clean)>30:
      tmplist.append(clean)
    else:
      continue
  sentsClean22.append(tmplist)

In [ ]:
preds20=[]

for idx, sent in enumerate(sentsClean20):
  if idx%10==0:
    print(idx)
  tmplist=[]
  for s in sent:
    prediction = classifier(s)
    tmplist.append(prediction[0])
  preds20.append(tmplist)

0


In [ ]:
preds22=[]

for idx, sent in enumerate(sentsClean22):
  if idx%10==0:
    print(idx)
  tmplist=[]
  for s in sent:
    prediction = classifier(s)
    tmplist.append(prediction[0])
  preds22.append(tmplist)

0


In [ ]:
li20=[]
for pred in preds20:
  tmplist=[]
  for p in pred:
    tmp=pd.DataFrame(p).set_index('label').T
    tmplist.append(tmp)
  tmpc=pd.concat(tmplist)
  res=tmpc.mean().reset_index()
  li20.append(res)

In [ ]:
li22=[]
for pred in preds22:
  tmplist=[]
  for p in pred:
    tmp=pd.DataFrame(p).set_index('label').T
    tmplist.append(tmp)
  tmpc=pd.concat(tmplist)
  res=tmpc.mean().reset_index()
  li22.append(res)

In [ ]:
li20[0]

,label,0
0,joy,0.551638
1,sadness,0.129092
2,fear,0.185671
3,anger,0.111581
4,love,0.002966
5,surprise,0.019053


In [ ]:
tmplist=[]
for l in li20:
  tmp=pd.DataFrame(l).set_index('label').T
  tmplist.append(tmp)
tmpc=pd.concat(tmplist)
res20=tmpc.mean().reset_index()

In [ ]:
tmplist=[]
for l in li22:
  tmp=pd.DataFrame(l).set_index('label').T
  tmplist.append(tmp)
tmpc=pd.concat(tmplist)
res22=tmpc.mean().reset_index()

In [ ]:
res20

,label,0
0,joy,0.653907
1,sadness,0.067563
2,fear,0.151201
3,anger,0.115159
4,love,0.005283
5,surprise,0.006887


In [ ]:
res22

,label,0
0,joy,0.688175
1,fear,0.147459
2,anger,0.100087
3,sadness,0.057262
4,surprise,0.004063
5,love,0.002954


In [ ]:
r20=res20.set_index('label').T.reset_index()

In [ ]:
r22=res22.set_index('label').T.reset_index()

In [ ]:
r20['index']='2020'

In [ ]:
r22['index']='2022'

In [ ]:
r=pd.concat([r20,r22])

In [ ]:
r.reset_index(inplace=True)

In [ ]:
rl=r.melt(id_vars=['index'], value_vars=['joy','fear','anger'])

In [ ]:
rl.rename(columns={'label':'Emotions detection: texts about AI technology','value':'Sentiment score','index':'Year'},inplace=True)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(rl, x='Emotions detection: texts about AI technology', y='Sentiment score',color='Year',barmode='group')

In [ ]:
fig